Instructions for Running This Notebook:
- Do not `Run All`.
- Instead, follow these steps: 
    1. Run all cells up to, but not including, the section [Modeling](#modeling)
    2. Next, execute the last cell within the section [Data Transformation](#data-transformation)
    3. Finally, run the remaining cells in the notebook

# Introduction

## [Overview](https://www.kaggle.com/competitions/widsdatathon2024-challenge1/overview)
- Objective: Develop a model to predict if patients received a metastatic cancer diagnosis within 90 days of screening (i.e., `DiagPeriodL90D`) using a unique oncology dataset.
- Motivation: 
    - Metastatic TNBC is considered the most aggressive TNBC and requires most urgent and timely treatment. Unnecessary delays in diagnosis and subsequent treatment can have devastating effects in these difficult cancers. Differences in the wait time to get treatment is a good proxy for disparities in healthcare access.
    - The primary goal of building these models is to detect relationships between demographics of the patient with the likelihood of getting timely treatment. The secondary goal is to see if environmental hazards impact proper diagnosis and treatment.
- Dataset
    - Source: Provided by Gilead Sciences, originating from Health Verity and enriched with third-party geo-demographic data and zip code level toxicology data from NASA/Columbia University.
    - Content: Information about demographics, diagnosis, treatment options, and insurance for patients diagnosed with breast cancer from 2015-2018.
    - Highlighted Features:
        - Demographics (e.g., age, gender, race, ...)
        - Diagnosis and treatment details (e.g., breast cancer diagnosis code, metastatic cancer diagnosis code, metastatic cancer treatments, ...)
        - Insurance information
        - Geo (zip-code level) demographic data (e.g, income, education, rent, race, poverty, ...)
        - Toxic air quality (zip-code level) data (e.g., Ozone, PM25 and NO2, ...)
- Evaluation
    - Metric: Area Under the Receiver Operating Characteristic (ROC) Curve (AUC-ROC).
    - Leaderboard:
        - During the competition: 51% of the test data.
        - Final standings: 49% of the test data.
- Submission Format
    - File Format: CSV
    - Columns:
        - `patient_id` (integer)
        - `DiagPeriodL90D` (percentage)
    - Example:
        ```
        patient_id,DiagPeriodL90D
        372069,.5
        981264,.5
        ```

## [Dataset](https://www.kaggle.com/competitions/widsdatathon2024-challenge1/data)
Roughly 18k records, each corresponds to a single patient and her Diagnosis Period
- `training.csv`
    - 12906 records
    - 83 columns, the last column is `DiagPeriodL90D` (int64)
- `test.csv`
    - 5792 records
    - 82 columns

# Setup

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

In [ ]:
from collections import Counter
import pickle
import random
from typing import List

from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from xgboost import XGBClassifier
from IPython.display import display

from utils.df_helpers import (
    find_duplicates, get_value_counts, get_combined_value_counts, highlight_diff, 
    highlight_nan, plot_category_distribution, plot_side_by_side_histogram_with_percentages
)
from utils.eval_helpers import (
    evaluate_binary_classifier, rank_features_by_importance
)

In [ ]:
random_state = 42
random.seed(random_state)
title_fontsize = 14
label_fontsize = 12
tick_fontsize = 10
text_fontsize = 10
pd.set_option("mode.copy_on_write", True)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
sns.set_theme(context="notebook", style="darkgrid", palette="deep", font="sans-serif", font_scale=1, color_codes=True, rc=None)

# Exploratory Data Analysis

In [ ]:
df_training = pd.read_csv('data/training.csv')
df_training.info()

<span style="color: #57b9ff;">
From the `dtypes` (NumPy data types: `np.float64`, `np.int64`; Python's built-in type: `object`), we know that the missing values in all columns can only be `numpy.nan`, i.e., `NaN`.
</span>

## Drop Duplicates

<span style="color: #57b9ff;">
There are some records where all attributes are identical except for `patient_id`. After removing these duplicates (keeping the first occurrence), 12,870 records remain.
</span>

In [ ]:
# find duplicates except the `patient_id` column
duplicates_df, columns_to_check = find_duplicates(df_training, 'patient_id')
first_occurrence_df = duplicates_df.drop_duplicates(subset=columns_to_check)
# print('All duplicated rows (including the first occurrence):')
# display(duplicates_df)
print(f'#Duplicated rows (excluding the first occurrence): {len(duplicates_df) - len(first_occurrence_df)}')

In [ ]:
# remove duplicates
df_training_without_duplicates = df_training.drop_duplicates(subset=columns_to_check, ignore_index=True)
print(f'Removed {len(df_training) - len(df_training_without_duplicates)} duplicated rows.')
print(f'Current #Patients: {len(df_training_without_duplicates)}')
df_training_without_duplicates.info()

## Inspect Columns

In [ ]:
target_column = 'DiagPeriodL90D'

### `DiagPeriodL90D`

<span style="color: #57b9ff;">
Approximately 38% of patients did not receive a metastatic cancer diagnosis within 90 days of screening.
</span>

In [ ]:
get_value_counts(df_training_without_duplicates, target_column)

In [ ]:
plot_category_distribution(df_training_without_duplicates, target_column, [(0, '>= 90 Days'), (1, '< 90 Days')], title_fontsize, tick_fontsize)

### `patient_race`

<span style="color: #57b9ff;">
Race information is unknown for approximately 50% of patients. Among the known cases, the only race that comprises more than 10% is White, at about 28%. Patients with unknown races are slightly more likely to be undiagnosed within 90 days of screening, while patients identified as White are slightly more likely to be diagnosed timely.
</span>

In [ ]:
highlight_nan(get_combined_value_counts(df_training_without_duplicates, 'patient_race', target_column, [0, 1]))

### `payer_type`

<span style="color: #57b9ff;">
Around 47% of patients have commercial insurance. The proportions of patients with Medicaid or Medicare are similar, each close to 20%. Patients with unknown insurance types are slightly more likely to be diagnosed timely, while those with commercial insurance are slightly more likely to be undiagnosed within 90 days of screening.
</span>

In [ ]:
highlight_nan(get_combined_value_counts(df_training_without_duplicates, 'payer_type', target_column, [0, 1]))

### `patient_state`

<span style="color: #57b9ff;">
The top 6 states where more than 5% of patients come from are CA (18.9%), TX (8.9%), NY (8.1%), MI (6.6%), IL (6.1%), and OH (5.9%). The tail of the distribution includes RI, NH, and MA, each at 0.00777%. Patients from some states, such as CA, MI, MN and CO, tend to receive more timely diagnoses compared to those from other states.
</span>

In [ ]:
highlight_nan(get_combined_value_counts(df_training_without_duplicates, 'patient_state', target_column, [0, 1]))

### `patient_zip3`

<span style="color: #57b9ff;">
There are 739 zip3 codes with patient distribution ranging from 0.00777% to 1.8%. There are no clear differences among zip3 codes in terms of receiving a timely diagnosis.
</span>

In [ ]:
patient_zip3_counts_df = get_combined_value_counts(df_training_without_duplicates, 'patient_zip3', target_column, [0, 1])
print(f'#Zip3 codes: {len(patient_zip3_counts_df)}')
highlight_nan(patient_zip3_counts_df.head(10))
# highlight_nan(patient_zip3_counts_df.tail(10))

### `patient_age`

<span style="color: #57b9ff;">
This dataset includes patients ranging from 18 to 91 years old, with more than 80% of the patients aged between 40 and 80. Patients older than 60 are slightly more likely to be diagnosed timely, while younger patients are slightly more likely to be missed.
</span>

In [ ]:
print(f'Min & max age: {df_training_without_duplicates['patient_age'].min()}, {df_training_without_duplicates['patient_age'].max()}')
age_boundries = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # bins: [10-20), [20-30), [30-40), [40-50), [50-60), [60-70), [70-80), [80-90), [90-100]
plot_side_by_side_histogram_with_percentages(df_training_without_duplicates, 'patient_age', target_column, [0, 1], age_boundries, title_fontsize, label_fontsize, text_fontsize, tick_fontsize)

### `patient_gender`

<span style="color: #57b9ff;">
All patients are female.
</span>

In [ ]:
highlight_nan(get_value_counts(df_training_without_duplicates, 'patient_gender'))

### `bmi`

<span style="color: #57b9ff;">
Nearly 70% of patients lack BMI information, and these patients are slightly more likely to receive a timely diagnosis compared to those with BMI data. For the available records, BMI ranges from 14.0 to 85.0. Most patients fall into the categories of 'Normal weight', 'Pre-obesity', and 'Obesity class I'. Patients with normal weights are slightly more likely to receive timely diagnoses, whereas those with higher BMIs are less likely to do so.
</span>

WHO Classification of BMI

| BMI Interval (kg/m²) | Category                |
|----------------------|-------------------------|
| < 18.5               | Underweight             |
| 18.5 - 24.9          | Normal weight           |
| 25 - 29.9            | Pre-obesity             |
| 30 - 34.9            | Obesity class I         |
| 35 - 39.9            | Obesity class II        |
| ≥ 40                 | Obesity class III       |

In [ ]:
bmi_df = df_training_without_duplicates['bmi'].isna().to_frame(name='is_bmi_missing')
highlight_nan(get_combined_value_counts(pd.merge(bmi_df, df_training_without_duplicates, how='left', left_index=True, right_index=True), 'is_bmi_missing', target_column, [0, 1]))

In [ ]:
bmi_boundries = [0, 18.5, 25, 30, 35, 40, 85]  # bins: [0, 18.5), [18.5, 25), [25, 30), [30, 35), [35, 40), [40, 85]
id_bmi_df = df_training_without_duplicates[['patient_id', 'bmi']].dropna()
id_target_df = df_training_without_duplicates[['patient_id', target_column]]
id_bmi_target_df = pd.merge(id_bmi_df, id_target_df, how='left', on='patient_id')
plot_side_by_side_histogram_with_percentages(id_bmi_target_df, 'bmi', target_column, [0, 1], bmi_boundries, title_fontsize, label_fontsize, text_fontsize, tick_fontsize)

### `breast_cancer_diagnosis_code`

<span style="color: #57b9ff;">
    <ol>
        <li>The number of ICD-10 patients is 3.3 times that of ICD-9 patients. The most common codes are 174.9 (15.3%), C50.911 (13.9%), C50.912 (13.3%), and C50.919 (11.4%). Many other codes have fewer than 10 patients.
        <li>Patients with a primary diagnosis code of 174.9 are more than 16 times as likely to remain undiagnosed within 90 days after screening. Generally, patients with ICD-9 codes are less likely to receive a timely diagnosis compared to those with ICD-10 codes.
        <li>ICD-10 codes offer more detailed diagnoses for breast cancer compared to ICD-9 codes. While it is possible to map ICD-9 codes to ICD-10 codes, this process can alter the distribution of diagnosis codes. Therefore, direct conversion is not performed. However, some available mappings include: C50.9|174.9, C50.41|174.4, C50.81|174.8, C50.21|174.2, C50.11|174.1, C50.51|174.5, C50.31|174.3, C50.01|174.6.
        <li>Some codes in the dataset (C50.929, C50.021, 175.9, C50.421) indicate breast cancer in males, which contradicts the information in the `patient_gender` column. These discrepancies suggest that there may be man-made errors present in the dataset, possibly introduced during data collection or data preparation. These records are not removed but require additional attention.
        <li>The code 198.81 represents a secondary malignant neoplasm of the breast, indicating that the cancer has metastasized to the breast from another part of the body. This differs from the other codes in the dataset, which represent primary breast cancer. Only one patient out of nine has `DiagPeriodL90D=1`. This definitely requires attention.
    </ol>
</span>

In [ ]:
is_icd10 = df_training_without_duplicates['breast_cancer_diagnosis_code'].str.startswith('C')
icd10_df = df_training_without_duplicates[is_icd10]
icd9_df = df_training_without_duplicates[~is_icd10]
print(f'The ratio of ICD-10 codes to ICD-9 codes: {len(icd10_df)/len(icd9_df):.1f}')

In [ ]:
code_counts_df = get_combined_value_counts(df_training_without_duplicates, 'breast_cancer_diagnosis_code', target_column, [0, 1])
code_desc_df = df_training_without_duplicates[['breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc']].drop_duplicates()
code_counts_desc_df = pd.merge(code_counts_df, code_desc_df, left_index=True, right_on='breast_cancer_diagnosis_code')
highlight_nan(code_counts_desc_df)

#### Edge Cases


In [ ]:
male_codes = ['C50929', 'C50021', '1759', 'C50421']
secondary_code = '19881'

In [ ]:
male_code_df = df_training_without_duplicates[df_training_without_duplicates['breast_cancer_diagnosis_code'].isin(male_codes)]
male_code_df[['patient_id', target_column]]

In [ ]:
secondary_code_df = df_training_without_duplicates[df_training_without_duplicates['breast_cancer_diagnosis_code'] == secondary_code]
secondary_code_df[['patient_id', target_column]]

### `breast_cancer_diagnosis_desc`

<span style="color: #57b9ff;">
    <ol>
        <li>Many words in the descriptions, such as 'Malignant'/'Malig', 'neoplasm'/'neopl', 'female', and 'breast', are not informative since they appear in almost every field. Removing these words can make the descriptions shorter and more concise. For example, the description 'Malignant neoplasm of breast (female), unspecified' can be reduced to 'unspecified' eventually.
        <li>Some words have short forms, and using a uniform representation can reduce unnecessary variations. For this purpose, 'unsp' will be replaced by 'unspecified', and 'ovrlp' will be replaced by 'overlapping'.
        <li>Each original description consists of 4 to 10 words. These descriptions specify the position of the neoplasm, such as the quadrant and side. After cleaning, the descriptions have been reduced to 1 to 7 words, with the single one-word description ('of') corresponding to C50.
        <li>Through further preprocessing, the text will be converted to lowercase, and common English stop words (e.g., 'of', 'and', etc.) will be removed. Punctuation and special characters will also be eliminated implicitly by the text encoder.
    </ol>
</span>

In [ ]:
def remove_tokens(text: str, tokens: List[str]) -> str:
    words = text.split()
    filtered_words = [word for word in words if word not in tokens]
    return ' '.join(filtered_words)

def replace_tokens(text: str, replacements: dict) -> str:
    words = text.split()
    replaced_words = [replacements.get(word, word) for word in words]
    return ' '.join(replaced_words)

In [ ]:
textual_column = 'breast_cancer_diagnosis_desc'
textual_column_cleaned = textual_column + '_cleaned'
tokens_to_remove = ['Malignant', 'malignant', 'Malig',
                    'neoplasm', 'neoplm', 
                    'female', '(female),',
                    'breast', 'breast,']
token_replacements = {
    'unsp': 'unspecified',
    'ovrlp': 'overlapping'
}

In [ ]:
code_counts_desc_df[textual_column_cleaned] = code_counts_desc_df[textual_column].apply(lambda x: remove_tokens(x, tokens_to_remove))
code_counts_desc_df[textual_column_cleaned] = code_counts_desc_df[textual_column_cleaned].apply(lambda x: replace_tokens(x, token_replacements))
desc_count = code_counts_desc_df[textual_column].apply(lambda text: len(text.split()))
desc_cleaned_count = code_counts_desc_df[textual_column_cleaned].apply(lambda text: len(text.split()))
print(f'Original #Words: min: {desc_count.min()}, max: {desc_count.max()}')
print(f'Current #Words: min: {desc_cleaned_count.min()}, max: {desc_cleaned_count.max()}')

In [ ]:
one_word_code = 'C50'
one_word_code_df = df_training_without_duplicates[df_training_without_duplicates['breast_cancer_diagnosis_code'] == one_word_code]
one_word_code_df[['patient_id', target_column]]

### `metastatic_cancer_diagnosis_code`

<span style="color: #57b9ff;">
    <ol>
        <li>All of the diagnosis codes are ICD-10 codes. The top 3 codes are C77.3(54.6%; Secondary and unspecified malignant neoplasm of axilla and upper limb lymph nodes), C79.51(14.2%; Secondary malignant neoplasm of bone), C77.9(5.9%; Secondary and unspecified malignant neoplasm of lymph node, unspecified).
        <li>Patients with code C77.3 are slightly more likely to be diagnosed timely.
        <li>Only 9 out of 43 diagnosis codes have metastatic first novel treatments, and these treatments occur in 24 out of all patients in the dataset (i.e., 0.186%). Among these patients, 8 were not diagnosed within 90 days after screening.
    </ol>
</span>

In [ ]:
highlight_nan(get_combined_value_counts(df_training_without_duplicates, 'metastatic_cancer_diagnosis_code', target_column, [0, 1]))

In [ ]:
metastatic_code_counts_df = get_value_counts(df_training_without_duplicates, 'metastatic_cancer_diagnosis_code')
metastatic_code_treatment_df = df_training_without_duplicates[['metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']].drop_duplicates().dropna()
metastatic_code_counts_treatment_df = pd.merge(metastatic_code_counts_df, metastatic_code_treatment_df, left_index=True, right_on='metastatic_cancer_diagnosis_code')
metastatic_code_counts_treatment_df

In [ ]:
patients_with_first_novel_treatments_df = df_training_without_duplicates[df_training_without_duplicates['metastatic_first_novel_treatment'].notna()]
patients_with_first_novel_treatments_df[['patient_id', target_column]]

### `metastatic_first_novel_treatment` & `metastatic_first_novel_treatment_type`

<span style="color: #57b9ff;">
There are only two metastatic first novel treatments: PEMBROLIZUMAB and OLAPARIB. Both of these treatments are of the type Antineoplastics.
</span>

In [ ]:
highlight_nan(get_combined_value_counts(df_training_without_duplicates, 'metastatic_first_novel_treatment', target_column, [0, 1]))

In [ ]:
highlight_nan(get_combined_value_counts(df_training_without_duplicates, 'metastatic_first_novel_treatment_type', target_column, [0, 1]))

### `Region`

<span style="color: #57b9ff;">
Patients from the South and Northeast are slightly more likely to remain undiagnosed within 90 days, whereas those from the West and Midwest are slightly more likely to receive a timely diagnosis.
</span>

In [ ]:
highlight_nan(get_combined_value_counts(df_training_without_duplicates, 'Region', target_column, [0, 1]))

### `Division`

<span style="color: #57b9ff;">
Patients from the Pacific, Mountain and West North Central divisions are slightly more likely to receive a timely diagnosis, compared to patients from other divisions.
</span>

In [ ]:
highlight_nan(get_combined_value_counts(df_training_without_duplicates, 'Division', target_column, [0, 1]))

### `population`, `density`, ..., `veteran`

<span style="color: #57b9ff;">
    <ol>
        <li>All patients have complete or partial geo- (zip-code level) demographic data, with the exception of patient ID 224030, whose zip3 code (332) is the only occurrence of this value in the dataset.
        <li>In addition to that single patient, three patients with the zip3 code 772 also have missing values in columns starting with 'family', 'income_household', 'home', 'rent', 'self_employed', 'farmer', 'poverty', and 'limited_english'. These three patients are the only ones with this specific zip3 code.
    </ol>
</span>

In [ ]:
na_population_df = df_training_without_duplicates[df_training_without_duplicates['population'].isna()]
na_population_zip3 = na_population_df['patient_zip3'].unique()
print(na_population_zip3)

In [ ]:
df_training_without_duplicates[df_training_without_duplicates['patient_zip3'] == na_population_zip3[0]]

In [ ]:
na_family_size_df = df_training_without_duplicates[df_training_without_duplicates['family_size'].isna()]
na_family_size_zip3 = na_family_size_df['patient_zip3'].unique()
print(na_family_size_zip3)

In [ ]:
df_training_without_duplicates[df_training_without_duplicates['patient_zip3'] == na_family_size_zip3[0]]

### `Ozone`, `PM25`, `N02`

<span style="color: #57b9ff;">
29 patients with the zip3 code 967, 968, 998, 995, 996, and 999 lack information about environmental hazards. These patients are the only ones with these specific zip3 codes.
</span>

In [ ]:
na_ozone_df = df_training_without_duplicates[df_training_without_duplicates['Ozone'].isna()]
na_ozone_zip3 = na_ozone_df['patient_zip3'].unique()
print(na_ozone_zip3)

In [ ]:
df_training_without_duplicates[df_training_without_duplicates['patient_zip3'].isin(na_ozone_zip3)]

### Summary

| Column & Values                  | **Timely**                            | **Delayed**                                                             |
|----------------------------------|---------------------------------------|-------------------------------------------------------------------------|
| patient_race                     | White                                 | nan                                                                     |
| payer_type                       | nan                                   | COMMERCIAL                                                              |
| patient_state                    | CA, MI, MN, CO                        | TX, NY, IL, VA, KY                                                      |
| patient_age                      | >60                                   | <=60                                                                    |
| bmi                              | nan, <25                              | not nan, >=25                                                           |
| breast_cancer_diagnosis_code     | ICD-10                                | 1749, ICD-9                                                             |
| metastatic_cancer_diagnosis_code | C773                                  | -                                                                       |
| Region                           | West, Midwest                         | South, Northeast                                                        |
| Division                         | Pacific, Mountain, West North Central | South Atlantic, Middle Atlantic, West South Central, East South Central |

## Collect Edge Cases

In [ ]:
edge_case_dfs = [male_code_df, secondary_code_df, one_word_code_df, na_family_size_df, na_ozone_df]  # na_population_df is a part of na_family_size_df
edge_case_notes = ['male code', 'secondary code', 'one word code', 'na family size', 'na ozone']

edge_cases = []
for df, note in zip(edge_case_dfs, edge_case_notes):
    extracted_df = df[[target_column]]
    extracted_df['note'] = note
    edge_cases.append(extracted_df)
df_edge_cases = pd.concat(edge_cases, ignore_index=False)

## Check Linear Correlation

<span style="color: #57b9ff;">
Among all originally numerical columns:
    <ol>
        <li>Slight positive correlation: e.g., patient_age, education_bachelors, patient_zip3,  income_individual_median, home_value
        <li>Slight negative correlation: e.g., education_less_highschool, widowed, income_household_25_to_35, health_uninsured, commute_time
        <li>No correlation: race_asian, rent_burden, divorced, N02, race_native, farmer, age_median, home_ownership, veteran, age_60s, never_married
    </ol>
</span>

In [ ]:
corr_df = df_training_without_duplicates.corr(numeric_only=True)
corr_df[target_column].sort_values(ascending=False)

# Data Preparation

### Data Transformation

In [ ]:
# add new features based on EDA
def add_columns(df):
    df['is_state_CA_MI_MN_CO'] = df['patient_state'].isin(['CA', 'MI', 'MN', 'CO'])
    df['is_state_TX_NY_IL_VA_KY'] = df['patient_state'].isin(['TX', 'NY', 'IL', 'VA', 'KY'])
    df['is_age_over_60'] = df['patient_age'] > 60
    df['is_bmi_under_25'] = df['bmi'] < 25
    df['is_diagnosis_code_ICD10'] = df['breast_cancer_diagnosis_code'].str.startswith('C')
    df['is_region_west_midwest'] = df['Region'].isin(['West', 'Midwest'])
    df['is_region_south_northeast'] = df['Region'].isin(['South', 'Northeast'])
    df['is_division_pacific_mountain_west_north_central'] = df['Division'].isin(['Pacific', 'Mountain', 'West North Central'])
    df['is_division_south_atlantic_middle_atlantic_west_south_central_east_south_central'] = df['Division'].isin(['South Atlantic', 'Middle Atlantic', 'West South Central', 'East South Central'])
    return df

In [ ]:
columns_to_remove = ['patient_id', 'patient_gender']
categorical_columns = ['patient_race', 'payer_type', 'patient_state', 'patient_zip3',  
                       'breast_cancer_diagnosis_code', 'Region', 'Division', 'metastatic_cancer_diagnosis_code', 
                       'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']
numerical_columns = df_training_without_duplicates.columns.difference([target_column] + 
                                                                      columns_to_remove + 
                                                                      categorical_columns + 
                                                                      [textual_column]).to_list()
unimportant_zip3 = []

In [ ]:
def transform(df, is_test_data=False, cat_encoder=None, text_encoder=None):
    if is_test_data:
        if not cat_encoder or not text_encoder:
            raise ValueError("Both cat_encoder and text_encoder must be provided when is_test_data is True.")

    # 1. add columns
    df_augmented = add_columns(df.copy())
    # 1.1 boolean -> int
    new_boolean_columns = [col for col in df_augmented.columns if col.startswith('is_')]
    for col in new_boolean_columns:
        df_augmented[col] = df_augmented[col].astype(int)
    
    # 2. remove columns
    df_reduced = df_augmented.drop(columns=columns_to_remove)


    # 3. categorical -> numerical
    # 3.1 encode categories
    if not is_test_data:
        cat_encoder = OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=2)
        onehot_matrix = cat_encoder.fit_transform(df_reduced[categorical_columns])
    else:
        onehot_matrix = cat_encoder.transform(df_reduced[categorical_columns])
    cat_feature_names = cat_encoder.get_feature_names_out()
    onehot_df = pd.DataFrame(onehot_matrix.toarray(), columns=cat_feature_names)
    df_no_cat = pd.concat([df_reduced, onehot_df], axis=1).drop(columns=categorical_columns + unimportant_zip3)

    # 4. fill NaN in numerical columns with 0.
    df_no_cat[numerical_columns] = df_no_cat[numerical_columns].fillna(0.)

    # 5. textual -> numerical
    # 5.1 clean tokens
    df_no_cat[textual_column_cleaned] = df_no_cat[textual_column].apply(lambda x: remove_tokens(x, tokens_to_remove))
    df_no_cat[textual_column_cleaned] = df_no_cat[textual_column_cleaned].apply(lambda x: replace_tokens(x, token_replacements))
    # 5.2 encode text
    if not is_test_data:
        text_encoder = TfidfVectorizer(lowercase=True, stop_words='english')
        tfidf_matrix = text_encoder.fit_transform(df_no_cat[textual_column_cleaned])
    else:
        tfidf_matrix = text_encoder.transform(df_no_cat[textual_column_cleaned])
    textual_feature_names = text_encoder.get_feature_names_out()
    prefix = 'tfidf_'
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[prefix + name for name in textual_feature_names])
    df_prepared = pd.concat([df_no_cat, tfidf_df], axis=1).drop(columns=[textual_column, textual_column_cleaned])    

    return df_prepared, cat_encoder, text_encoder

In [ ]:
df_training_prepared, cat_encoder, text_encoder = transform(df_training_without_duplicates)
df_training_prepared.info()

In [ ]:
# re-run this and all subsequent cells after the 'Feature Reduction' section has been executed for the first time
df_training_prepared.drop(columns=unimportant_zip3, inplace=True)
df_training_prepared.info()

### Data Partition

In [ ]:
# split data into input and output, train and validation
X = df_training_prepared.drop(target_column, axis=1)
y = df_training_prepared[target_column]

X_train, X_validation, y_train, y_validation, train_indices, validation_indices = train_test_split(
    X, y, np.arange(len(X)), test_size=0.2, random_state=random_state
)
print(f'Training target distribution (after data splitting): {Counter(y_train)}')
print(f'Validation target distribution (after data splitting): {Counter(y_validation)}')

In [ ]:
# retrieve original data corresponding to validation set
df_validation = df_training_without_duplicates.loc[validation_indices]

In [ ]:
# construct balanced validation set
X_validation_array = X_validation.to_numpy()
y_validation_array = y_validation.to_numpy()
all_one_indices = np.where(y_validation_array == 1)[0].tolist()
random_one_indices = random.sample(all_one_indices, Counter(y_validation)[0])
balanced_indicies = np.where(y_validation_array == 0)[0].tolist() + random_one_indices
X_validation_balanced = pd.DataFrame(X_validation_array[balanced_indicies, :], columns=X_validation.columns)
y_validation_balanced = pd.Series(y_validation_array[balanced_indicies], name=y_validation.name)
print(f'Validation target distribution (after target balancing): {Counter(y_validation_balanced)}')

In [ ]:
def determine_partition(index: int, train_indices: np.ndarray, validation_indices: np.ndarray) -> str:
    if index in train_indices:
        return 'train'
    elif index in validation_indices:
        return 'validation'
    else:
        raise ValueError(f"Index {index} does not exist in either train_indices or validation_indices")

In [ ]:
# find partitions of edge cases
df_edge_cases['partition'] = df_edge_cases.index.map(lambda idx: determine_partition(idx, train_indices, validation_indices))
df_edge_cases['probability'] = np.nan
df_edge_cases['prediction'] = np.nan
edge_case_validation_indices = df_edge_cases[df_edge_cases['partition'] == 'validation'].index.to_list()
edge_case_evaluation_indices = [np.where(validation_indices == idx)[0][0] for idx in edge_case_validation_indices]

### Feature Scaling

In [ ]:
# apply standardization (mean=0, variance=1) to the data for logistic regression
scaler = StandardScaler()
X_train_standard_scaled = scaler.fit_transform(X_train)
X_validation_standard_scaled = scaler.transform(X_validation)
X_validation_balanced_standard_scaled = scaler.transform(X_validation_balanced)

In [ ]:
# apply min-max scaling (min=0, max=1) to the data for neural network
scaler = MinMaxScaler()
X_train_minmax_scaled = scaler.fit_transform(X_train)
X_validation_minmax_scaled = scaler.transform(X_validation)

### Data Sampling

In [ ]:
# oversample data
ros = RandomOverSampler(random_state=random_state)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
X_train_standard_scaled_resampled, _ = ros.fit_resample(X_train_standard_scaled, y_train)
X_train_minmax_scaled_resampled, _ = ros.fit_resample(X_train_minmax_scaled, y_train)
print(f'Training target distribution (after data resampling): {Counter(y_train_resampled)}')

### Feature Reduction

In [ ]:
feature_names = X.columns

In [ ]:
def eval(model, x_val, y_val, x_val_balanced, y_val_balanced, df_edge_cases, edge_case_evaluation_indices, df_validation, target_column, feature_names=None):
    if feature_names is not None:
        rank_features_by_importance(feature_names, model.feature_importances_, True)
    print('Scores on Imbalanced Validation Set:')
    scores_imbalanced_val, df_edge_cases, df_validation = evaluate_binary_classifier(model, x_val, y_val, True, True, df_edge_cases, edge_case_evaluation_indices, True, df_validation)
    display(scores_imbalanced_val)
    print('Scores on Balanced Validation Set:')
    scores_balanced_val, _, _ = evaluate_binary_classifier(model, x_val_balanced, y_val_balanced)
    display(scores_balanced_val)
    print('Edge Cases:')
    display(highlight_diff(df_edge_cases[df_edge_cases['partition'] == 'validation'], target_column, 'prediction'))
    print('Samples with High Absolute Errors:')
    display(df_validation.head(10))


In [ ]:
# use L1 to get unimportant features
model = LogisticRegression(penalty="l1", C=0.01, class_weight='balanced', 
                           random_state=random_state, solver="liblinear", max_iter=100)
model.fit(X_train_standard_scaled, y_train)
feature_importance = rank_features_by_importance(feature_names, model.coef_[0])
unimportant_features = feature_importance[feature_importance['Importance'] == 0.]['Feature'].to_list()
eval(model, X_validation_standard_scaled, y_validation, X_validation_balanced_standard_scaled, y_validation_balanced, df_edge_cases, edge_case_evaluation_indices, df_validation, target_column)

In [ ]:
if len(unimportant_zip3) == 0:
    unimportant_zip3 = [f for f in unimportant_features if f.startswith('patient_zip3') and len(f.split('_')[2]) == 3]

# Modeling

## Logistic Regression

<span style="color: #57b9ff;">
    <ol>
        <li>Resampled data was not used due to a degradation in ROC-AUC scores. Instead, setting `class_weight='balanced'` slightly improved the scores for all penalties.
        <li>The best-performing model uses L1 as penalty.
     </ol>
</span>

### L1

In [ ]:
model = LogisticRegression(penalty="l1", C=0.01, class_weight='balanced', 
                           random_state=random_state, solver="liblinear", max_iter=100)
model.fit(X_train_standard_scaled, y_train)
feature_importance = rank_features_by_importance(feature_names, model.coef_[0])
print(f'Top 10 Features with Positive Coefficients: \n{feature_importance.head(10)['Feature'].to_list()}')
print(f'Top 10 Features with Negative Coefficients: \n{feature_importance.tail(10)['Feature'].to_list()}')
print(f'All Features with Zero Coefficients: \n{feature_importance[feature_importance['Importance'] == 0.]['Feature'].to_list()}\n')
eval(model, X_validation_standard_scaled, y_validation, X_validation_balanced_standard_scaled, y_validation_balanced, df_edge_cases, edge_case_evaluation_indices, df_validation, target_column)

### L2

In [ ]:
model = LogisticRegression(penalty="l2", C=0.001, class_weight='balanced', 
                           random_state=random_state, solver="liblinear", max_iter=100)
model.fit(X_train_standard_scaled, y_train)
feature_importance = rank_features_by_importance(feature_names, model.coef_[0])
print(f'Top 10 Features with Positive Coefficients: \n{feature_importance.head(10)['Feature'].to_list()}')
print(f'Top 10 Features with Negative Coefficients: \n{feature_importance.tail(10)['Feature'].to_list()}')
eval(model, X_validation_standard_scaled, y_validation, X_validation_balanced_standard_scaled, y_validation_balanced, df_edge_cases, edge_case_evaluation_indices, df_validation, target_column)

### Elasticnet

In [ ]:
model = LogisticRegression(penalty="elasticnet", C=0.005, class_weight='balanced', 
                           random_state=random_state, solver="saga", max_iter=500, l1_ratio=0.5)
model.fit(X_train_standard_scaled, y_train)
feature_importance = rank_features_by_importance(feature_names, model.coef_[0])
print(f'Top 10 Features with Positive Coefficients: \n{feature_importance.head(10)['Feature'].to_list()}')
print(f'Top 10 Features with Negative Coefficients: \n{feature_importance.tail(10)['Feature'].to_list()}')
eval(model, X_validation_standard_scaled, y_validation, X_validation_balanced_standard_scaled, y_validation_balanced, df_edge_cases, edge_case_evaluation_indices, df_validation, target_column)

## Decision Tree

<span style="color: #57b9ff;">
Again, not using resampled data results in improved performance.
</span>

In [ ]:
model = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=random_state)
model.fit(X_train, y_train)
print(f'Depth of the tree: {model.get_depth()}')
print(f'Number of leaves: {model.get_n_leaves()}')
eval(model, X_validation, y_validation, X_validation_balanced, y_validation_balanced, df_edge_cases, edge_case_evaluation_indices, df_validation, target_column, feature_names)

In [ ]:
plt.figure(figsize=(100, 50))
plot_tree(model, feature_names=feature_names, filled=True)
plt.show()

## Random Forest

<span style="color: #57b9ff;">
The model performs best when `class_weight=None` and all samples are used to train each base estimator without resampling the data.
</span>

In [ ]:
model = RandomForestClassifier(n_estimators=250, criterion='gini', max_depth=5, 
                               oob_score=True, random_state=random_state, 
                               class_weight=None, max_samples=None)
model.fit(X_train, y_train)
print(f'Out-of-bag score: {model.oob_score_}')
eval(model, X_validation, y_validation, X_validation_balanced, y_validation_balanced, df_edge_cases, edge_case_evaluation_indices, df_validation, target_column, feature_names)

## XGBoost

<span style="color: #57b9ff;">
    <ol>
        <li>Again, not using resampled data works better.
        <li>It performs better than the other models.
     </ol>
</span>

In [ ]:
model = XGBClassifier(n_estimators=90, max_depth=4, learning_rate=0.005, 
                      objective="binary:logistic", booster="gbtree", 
                      reg_alpha=4.0, reg_lambda=1.5, random_state=random_state)
model.fit(X_train, y_train)
eval(model, X_validation, y_validation, X_validation_balanced, y_validation_balanced, df_edge_cases, edge_case_evaluation_indices, df_validation, target_column, feature_names)

## Neural Network

In [ ]:
with open('X_train_minmax_scaled.pkl', 'wb') as file:
    pickle.dump(X_train_minmax_scaled, file)
with open('X_train_minmax_scaled_resampled.pkl', 'wb') as file:
    pickle.dump(X_train_minmax_scaled_resampled, file)
with open('X_validation_minmax_scaled.pkl', 'wb') as file:
    pickle.dump(X_validation_minmax_scaled, file)
with open('y_train.pkl', 'wb') as file:
    pickle.dump(y_train.values, file)
with open('y_train_resampled.pkl', 'wb') as file:
    pickle.dump(y_train_resampled.values, file)
with open('y_validation.pkl', 'wb') as file:
    pickle.dump(y_validation.values, file)

The code and results are in the notebook [`widsdatathon2024-challenge1_nn.ipynb`](widsdatathon2024-challenge1_nn.ipynb).

# Testing

In [ ]:
# retrain on all training data
model = XGBClassifier(n_estimators=90, max_depth=4, learning_rate=0.005, 
                      objective="binary:logistic", booster="gbtree", 
                      reg_alpha=4.0, reg_lambda=1.5, random_state=random_state)
model.fit(X, y)

In [ ]:
# prepare test data
df_test = pd.read_csv('data/test.csv')
df_test_prepared, _, _ = transform(df_test, True, cat_encoder, text_encoder)
df_test_prepared.info()

In [ ]:
# get result
result = model.predict_proba(df_test_prepared)[:, 1]
df_result = pd.DataFrame({
    'patient_id': df_test['patient_id'],
    'DiagPeriodL90D': result
})
df_result.to_csv('test_result.csv', index=False)

# Summary

Key principles applied in this project include:
- Retain records with missing values (as the pattern of such data absences might reflect specific characteristics of patients that could be relevant to the target prediction)
- Tune hyperparameters only using a self-constructed validation set (reserving the test data for evaluation only after the best model has been determined)

Results:

| Positive                                        | Negative                                    |
|-------------------------------------------------|---------------------------------------------|
| `is_diagnosis_code_ICD10`                       | -                                           |
| `patient_age`                                   | -                                           |
| -                                               | `metastatic_cancer_diagnosis_code_C7981`    |
| `is_state_CA_MI_MN_CO`                          | -                                           |
| `payer_type_nan`                                | -                                           |
| -                                               | `health_uninsured`                          |
| `breast_cancer_diagnosis_code_[C50811, C50011]` | `breast_cancer_diagnosis_code_[1744, 1749]` |
| `is_region_west_midwest`                        | -                                           |
| -                                               | `patient_state_VA`                          |
| `patient_zip3_902`                              | `patient_zip3_[948, 130, 928, 957]`         |
| -                                               | `patient_race_Black`                        |
| `tfidf_[right, left, overlapping, site]`        | `tfdif_unspecified`                         |

- Based on the important features identified across each model, the following are nearly universal: `is_diagnosis_code_ICD10`, `patient_age`, and `metastatic_cancer_diagnosis_code_C7981`. The table above shows features that hold high importance in at least two models, categorized by their impact on timely diagnosis—whether positive (larger values indicate more timely diagnosis) or negative (larger values indicate less timely diagnosis). 

- The results align closely with the findings from the exploratory data analysis (EDA). Features such as `is_diagnosis_code_ICD10`, `is_state_CA_MI_MN_CO`, `payer_type_nan`, and `is_region_west_midwest` confirm expectations. Additionally, the models have identified new significant features, including `health_uninsured`, `patient_state_VA`, and `patient_race_Black`. The models also highlight more granular elements, such as specific zip3 codes, diagnosis codes, and description tokens, which are consistent with EDA insights. For instance, tokens like `right`, `left`, `overlapping`, and `site` are uniquely associated with ICD-10 codes.

- The highest ROC-AUC score is achieved by the XGBoost model, with scores of 0.801685 on the imbalanced validation set and 0.802429 on the balanced validation set. However, the recall for class 0 (DiagPeriodL90D=0) is low, indicating that the model tends to predict more patients as being diagnosed timely, likely due to the imbalanced training data. Resampling did not improve this aspect.

- Several geodemographic factors influencing diagnosis wait times highlight disparities in healthcare access. These factors include specific regions (West, Midwest), states (CA, MI, MN, CO), and zip3 codes (902, 948, 130, 928, 957). Regarding environmental hazards, while Ozone, PM25, and NO2 have marginal impacts on diagnosis timing, Ozone shows a slightly greater influence compared to the others.

Areas for further exploration:
- Alternative text encoders
- Different imputation methods for handling NaNs in numerical columns